In [8]:
import os
import pandas as pd
import numpy as np
import csv
import re

In [9]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
#读入数据
group_1_del = pd.read_csv('D:\my_research\gensim_group\group_1\group_1_final.csv',encoding='utf-8')
group_1_del.columns = ['0','content','date','time','user_id','cut_content','del']
group_1_del = group_1_del.drop(columns='0')
group_1_del = group_1_del.dropna(axis=0, how='any')
group_1_del.head()

,content,date,time,user_id,cut_content,del
0,#上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...,2022-04-24,2022-04-24 08:33:43,1000074972,# 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 # 上海 返乡 潮 开...,疫情 民生 一刻 返乡 返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮...
1,#上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...,2022-04-23,2022-04-23 05:46:14,1000074972,# 上海 疫情 ## 上海 民生 ## 上海 现状 # . 上海 的 现状 为什么 会 ...,疫情 民生 现状 现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 ...
2,#上海疫情##上海民生# 无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...,2022-04-22,2022-04-22 11:47:31,1000074972,# 上海 疫情 ## 上海 民生 # 无间道 ， 罗生门 ， 可能 ...,疫情 民生 无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼...
3,#上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。,2022-04-22,2022-04-22 09:41:33,1000074972,# 上海 民生 ## 上海 疫情 ## 上海 团长 # 还是 你 牛 逼 呀 ， 团长 ...,民生 疫情 团长 牛 逼 团长 开个 车照 送 封控 小区
4,明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...,2022-04-21,2022-04-21 08:54:29,1000074972,明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...,明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇 市场监管 江 丄 孤 ...


In [11]:
import pyLDAvis.sklearn
import pyLDAvis
import pyLDAvis.gensim

In [12]:
#完善停用词表
#new = (['顾魏','特别','越来越','好像','我要','刚刚','天天','林之校','啊啊啊','王一博','翟潇闻','真的','哈哈哈','感觉',
#       '不到','一点','呜呜','哈哈哈哈','好好','这是','时代','少年','余生','指教','本来','晚上','天天','不想','玖',
#        '团肖战','代言人','马嘉祺','尹浩宇','热巴','迪丽','刘宇宁','摩登','兄弟','周杰伦','新歌','余宇涵','原创',
#       '哔哩','.','两个','左航','周边','一善','太','岁','之恋','舞台','诶','好帅','好好看','生日快乐','美美',
#       '卧槽','演唱会','一条','歌','救命','救','想','吃','做','说','月','买','天','号','明天','小时','分钟','夜',
#       '穿','忘','祝','面','甜','咸','日','团','菜','愿','挺','明明','加','行','再也','不知','玩','挂','正','刷',
#        '真','有没有','写','打卡','越','站','走','画','风','路','想到','回','王','心','话','掉','搞','干',
#        '个头','  ','包','跳','图','份','开','字','咯','长','草','新','品牌','请','剧','哥哥','块','这部','超',
#       '破','超话','祝','那种','%','服','周','店','圈','哒','语','高','页','一只','刘','名','度','达','也许',
#       '综艺','先','亿','倒','水','元','拉','黑','宋亚轩','姐姐','嗨','受','录','秒','戏','俩','只会','手','焕新','聚力',
#       '耀','荣耀','大会','视界','滚','假','气','派','中','少','视界','杨','女团','至少','额','算是','花','再也','帅',
#        '生','之间','确实','耶','头','丢','易','烊','千玺','热','搜','道','灿烂','星汉','邓紫棋','棒','称','沉香',
#       '如屑','集','毛','算',' ',' ','|','|','点','年','里','微光','喵','徐坤'])

In [13]:
#aaa = []
#for line in group_1_del['del']:
#    line_u = ' '.join(str(line).split())
#    line_uu = line_u.split()#将一个元素按空格拆分成多个元素
#    outstr = []
#    for words in line_uu:
#        if words not in new:
            #if words in ['n','v','a','r','i']:
#                outstr.append(words)
#    aaa.append(outstr)

In [14]:
def clean(text):
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"\[\S+\]", "", text)      # 去除表情符号
    text = re.sub(r"#\S+#", "", text)      # 保留话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)       # 去除网址
    text = text.replace('转发微博','') # 去除无意义的词语
    text = text.replace('网页链接','')
    text = text.replace('的微博视频','')
    text = text.replace('妞妞端午花草','')
    text = text.replace('bluepoint2006','')
    text = text.replace('上海','')
    text = re.sub(r"\s+", " ", text) # 合并正文中过多的空格
    return text.strip()

In [15]:
count = []
for i in group_1_del['content']:
    line_u = str(i)
    line_uu = clean(line_u)
    count.append(line_uu)  

In [16]:
res = list ( filter ( None , count))

In [17]:
len(res)

281518

In [18]:
list1=res
list2=[]
for i in list1:
    if i not in list2:
        list2.append(i)
print(list2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
len(list2)

277606

In [20]:
import jieba
import jieba.posseg as pseg

In [21]:
def cut(x):
    return (word for word, flag in pseg.cut(x) if flag in ['n'])#副词

texts = [cut(a) for a in list2]

In [22]:
text = list ( filter ( None ,texts))

In [23]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [24]:
dictionary = corpora.Dictionary(text)
dictionary.filter_extremes(no_below=2, no_above=0.8, keep_n=None)
corpus = [dictionary.doc2bow(tmp) for tmp in text]

Building prefix dict from the default dictionary ...
2022-12-07 22:13:04,888 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
2022-12-07 22:13:04,888 : DEBUG : Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 1.147 seconds.
2022-12-07 22:13:06,035 : DEBUG : Loading model cost 1.147 seconds.
Prefix dict has been built successfully.
2022-12-07 22:13:06,035 : DEBUG : Prefix dict has been built successfully.


In [25]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [ ]:
# 困惑度的XY坐标
x_perplexity = [] 
y_perplexity = [] 
# 一致性的XY坐标
x_coherence = [] 
y_coherence = [] 
#id2word：就是上边构造的dictionary字典
#passes：模型的训练次数
for i in range(1, 31):     
    # 循环生成主题数为i的模型， passes代表模型训练次数，数据量太大了，搞小一点。     
    lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=i, id2word=dictionary, passes=10)     
    # 计算当前模型困惑度
    cur_perplexity = lda.log_perplexity(corpus_tfidf)     
    # 构造横纵坐标轴数据
    x_perplexity.append(i)
    y_perplexity.append(cur_perplexity) 
    # 计算一致性
    cv_tmp = CoherenceModel(model=lda, texts=text, dictionary=dictionary, coherence='c_v') 
    x_coherence.append(i) 
    y_coherence.append(cv_tmp.get_coherence()) 

D:\Program_Files\anaconda\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:202: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
D:\Program_Files\anaconda\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:203: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
D:\Program_Files\anaconda\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:198: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs


In [ ]:
# 绘制一致性折线图 
plt.plot(x_coherence, y_coherence) 
plt.xlabel('num topics') 
plt.ylabel('coherence score') 
plt.legend(('coherence_values'), loc='best') 
plt.savefig('D:\my_research\gensim_group\new_coherenceLine_1.jpg') 
plt.show() 

In [ ]:
cohen = pd.DataFrame(y_coherence, x_coherence)
cohen.to_csv('D:\my_research\gensim_group\group_coherence_1.csv',encoding="utf-8_sig") 

In [ ]:
perp =  pd.DataFrame(y_perplexity, x_perplexity)
perp.to_csv('D:\my research\gensim_group\group_perplexity_1.csv',encoding="utf-8_sig") 

In [ ]:
#lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=6, id2word=dictionary, passes=10)

In [ ]:
#pyLDAvis.enable_notebook()
#data = pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary)
#让可视化可以在notebook内显示
#pyLDAvis.show(data)